In [1]:
import pandas as pd
import glob # to scan files from a folder and get it in a list of names with locations
from functools import reduce #to be able to merge the items within a list

The input files have three tabs and data from all three tabs can be used in the model. 
There is seperate excel for each quarter data. Glob function is used to scan the name of files in folder and they are imported one by one
Import data from each tab one by one and finally colate into one final file 

In [2]:
# factor 2 -US plant capacity Utilization
# using a function to import all files together
# below steps will get the list of each quarterly file in both xls and xlsx format into one list

path = r'C:\\Users\\z0047cfr\\Project Work\\Factory Demand Model\\Input Data\\US plant capacity Utilization' # use your path
xls_files = glob.glob(path + "/*.xls")
xlsx_files = glob.glob(path + "/*.xlsx")
all_files = xls_files + xlsx_files # list of final xls and xlsx files that make up the total quarterly files

In [3]:
#full utilization rates


li = []

for filename in all_files: # import data
    F2_3 = pd.read_excel(filename, index_col=None, sheet_name='FULL Utilization Rates', skiprows = 4, usecols = [0,3], names = ['naics',  'util_rate'], converters={'naics':str,'naics':str})
      
    # since the data for each quarter is in a seperate file, the file name is used to identify quarter and input it in varname
    year = filename[99:103]
    qtr = filename[120:122]
    qtr_yr = qtr + '_' + year 
    names = F2_3.columns.tolist()
    names[names.index('util_rate')] = qtr_yr
    F2_3.columns = names
    
    #remove spacings
    F2_3 = F2_3.applymap(lambda x: x.strip() if isinstance(x, str) else x)  
    #F2_3.apply(lambda x: x.str.replace(',',' '))
    
    # values of NAICS slightly changed after a time period. Correcting below to ensure data is appended properly
    F2_3["naics"]= F2_3["naics"].str.replace("3323, 7, 8, 9", "3323,7-9", case = False) 
    F2_3["naics"]= F2_3["naics"].str.replace("3365, 6, 9", "3365-9", case = False) 
    F2_3["naics"]= F2_3["naics"].str.replace("3311, 2", "3311,2", case = False) 
    
    naics_list = ['31-33', '3311,2', '3311, 2', '3323,7-9', '3323, 7, 8, 9', '3335', '3365-9', '3365, 6, 9']
    F2_3 = F2_3[F2_3['naics'].isin(naics_list)]    
    
    li.append(F2_3) # appending the files into one list

#after seperate datasets are created for eachquarterly file, the below step will merge it together on the time period
F2_3_merged = reduce(lambda  left,right: pd.merge(left,right,on=['naics'],
                                            how='outer'), li)
    

#F2_3.head()
#F2_3.columns
#li[1]
#frame
F2_3_merged

,naics,q1_2014,q2_2014,q3_2014,q4_2014,q1_2015,q2_2015,q3_2015,q4_2015,q1_2016,...,q4_2017,q1_2018,q2_2018,q3_2018,q4_2018,q1_2019,q2_2019,q3_2019,q4_2019,q1_2020
0,31-33,70.682764,73.289174,73.542828,71.297958,69.861024,74.1,72.8,70.6,71.7,...,72.3,73.9,75.0,76,73.8,70.6,74.2,72.3,71.4,71.6
1,"3311,2",75.584535,74.605062,79.330705,73.866712,68.511414,71.7,72.2,63.3,69.6,...,73.3,75.0,74.5,75.8,77.7,77.5,75.3,73.7,70.3,75.3
2,"3323,7-9",65.873075,69.774459,61.465898,67.332137,68.686401,63.8,63.4,62.6,64.5,...,65.4,67.6,71.7,66.5,68.9,66.8,69.7,69.7,67.5,66.1
3,3335,60.304354,65.474817,60.850412,62.517045,63.370406,73.9,69.1,72.2,65.3,...,69.1,70.6,72.1,71.5,74.9,70.3,79.6,74.9,73.6,72.8
4,3365-9,56.267607,50.067875,65.613286,48.202542,51.422860,68.7,64.0,54.0,58.1,...,54.6,56.2,57.5,56.1,54.7,57.9,58.4,59.7,54.9,55.7


In [4]:
#Transpose
# transposing to get data in the modelling format
F2_3_=F2_3_merged.T
F2_3_=F2_3_[1:]
F2_3_['month'] = F2_3_.index
F2_3_.columns = ['FU_Avg_Plant_hpw_all_mfg_Q', 'FU_Avg_Plant_hpw_metal_mfg_Q', 'FU_Avg_Plant_hpw_fab_metal_mfg_Q',
                 'FU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q', 'FU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q', 'month']
F2_3_.head()


,FU_Avg_Plant_hpw_all_mfg_Q,FU_Avg_Plant_hpw_metal_mfg_Q,FU_Avg_Plant_hpw_fab_metal_mfg_Q,FU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q,FU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q,month
q1_2014,70.6828,75.5845,65.8731,60.3044,56.2676,q1_2014
q2_2014,73.2892,74.6051,69.7745,65.4748,50.0679,q2_2014
q3_2014,73.5428,79.3307,61.4659,60.8504,65.6133,q3_2014
q4_2014,71.298,73.8667,67.3321,62.517,48.2025,q4_2014
q1_2015,69.861,68.5114,68.6864,63.3704,51.4229,q1_2015


In [5]:
# handy list of NAICs used for factor 2of3 and 3of3
#naics	desc	q1_2020	var_name
#1	31-33	All manufacturing 1/	56	A_Plant_hpw_31-33_All manufacturing 1/
#56	3311,2	Iron and steel mills and ferroalloy mfg.; and ...	72.7	A_Plant_hpw_3311,2_Iron and steel mills and fe...
#61	3323,7-9	All other fabricated metal product mfg.	48.8	A_Plant_hpw_3323,7-9_All other fabricated meta...
#66	3335	Metalworking machinery mfg.	62.6	A_Plant_hpw_3335_Metalworking machinery mfg.
#84	3365-9	Railroad rolling stock, ship, boat, and other ...	43.5	A_Plant_hpw_3365-9_Railroad rolling stock, shi...

In [6]:
# emergency utilization rates
li_1 = []

for filename in all_files:
    F2_2 = pd.read_excel(filename, index_col=None, sheet_name='EMERGENCY Utilization Rates', skiprows = 4, usecols = [0,3], names = ['naics',  'util_rate'], converters={'naics':str,'naics':str})
    
    # since the data for each quarter is in a seperate file, the file name is used to identify quarter and input it in varname  
    year = filename[99:103]
    qtr = filename[120:122]
    qtr_yr = qtr + '_' + year 

    names = F2_2.columns.tolist()
    names[names.index('util_rate')] = qtr_yr
    F2_2.columns = names
    
    # values of NAICS slightly changed after a time period. Correcting below to ensure data is appended properly
    F2_2 = F2_2.applymap(lambda x: x.strip() if isinstance(x, str) else x)  
    #F2_2.apply(lambda x: x.str.replace(',',' '))
    F2_2["naics"]= F2_2["naics"].str.replace("3323, 7, 8, 9", "3323,7-9", case = False) 
    F2_2["naics"]= F2_2["naics"].str.replace("3365, 6, 9", "3365-9", case = False) 
    F2_2["naics"]= F2_2["naics"].str.replace("3311, 2", "3311,2", case = False) 
    
    naics_list = ['31-33', '3311,2', '3311, 2', '3323,7-9', '3323, 7, 8, 9', '3335', '3365-9', '3365, 6, 9']
    F2_2 = F2_2[F2_2['naics'].isin(naics_list)]
    #F2_2.reset_index(drop=True)
    #F2_2.set_index('naics')
    
    #F2_2["utilizn_rate"] = 'A_Plant_hpw_' + F2_2["naics"] + '_' + F2_2["desc"]
    
    
    li_1.append(F2_2)# appending the files into one list

#after seperate datasets are created for eachquarterly file, the below step will merge it together on the time period
F2_2_merged = reduce(lambda  left,right: pd.merge(left,right,on=['naics'],
                                            how='outer'), li_1)
    
#frame = pd.concat(li, axis=1, ignore_index=True)
#frame.filter(regex='^q',axis=1).head()
#pd.merge(li,F2_2, on='naics')

#F2_2.head()
#F2_2.columns
#li[1]
#frame
F2_2_merged

,naics,q1_2014,q2_2014,q3_2014,q4_2014,q1_2015,q2_2015,q3_2015,q4_2015,q1_2016,...,q4_2017,q1_2018,q2_2018,q3_2018,q4_2018,q1_2019,q2_2019,q3_2019,q4_2019,q1_2020
0,31-33,55.348634,58.027583,57.935928,55.404055,54.193642,56.8,56.2,54.6,56.2,...,57.3,58.9,60.1,61.2,59.5,58.4,59.5,57.7,56.8,56
1,"3311,2",65.740307,66.513904,72.002702,65.638181,62.117589,64.7,65.6,56.9,63.5,...,68.6,70.4,68.7,69.5,70.1,74.6,72,70.1,67.1,72.7
2,"3323,7-9",47.554324,50.311318,46.494843,49.068048,48.825964,41.7,38.9,39.0,41.3,...,46.3,48.2,52.9,49.4,49.4,49.5,50.4,47.2,45.6,48.8
3,3335,45.809750,51.491970,47.626614,49.597980,47.938925,53,45.9,49.2,42.8,...,50.6,53.8,48.9,53.7,62.3,53.5,65.1,62.1,61.2,62.6
4,3365-9,43.340274,40.153470,49.700563,36.663219,39.073273,52.7,48.4,43.1,45.1,...,42.1,42.8,44.0,40.9,43.7,45.3,40.9,44.5,42.1,43.5


In [7]:
#Transpose
F2_2_=F2_2_merged.T
F2_2_=F2_2_[1:]
F2_2_['month'] = F2_2_.index
F2_2_.columns = ['EU_Avg_Plant_hpw_all_mfg_Q', 'EU_Avg_Plant_hpw_metal_mfg_Q', 'EU_Avg_Plant_hpw_fab_metal_mfg_Q',
                 'EU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q', 'EU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q', 'month']
F2_2_.head()
#F2_2_.dtypes
#F2_2_.columns

,EU_Avg_Plant_hpw_all_mfg_Q,EU_Avg_Plant_hpw_metal_mfg_Q,EU_Avg_Plant_hpw_fab_metal_mfg_Q,EU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q,EU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q,month
q1_2014,55.3486,65.7403,47.5543,45.8097,43.3403,q1_2014
q2_2014,58.0276,66.5139,50.3113,51.492,40.1535,q2_2014
q3_2014,57.9359,72.0027,46.4948,47.6266,49.7006,q3_2014
q4_2014,55.4041,65.6382,49.068,49.598,36.6632,q4_2014
q1_2015,54.1936,62.1176,48.826,47.9389,39.0733,q1_2015


In [8]:
# factor 2 -US plant capacity Utilization
# sheet 3 of 3
# using a function to import all files together
import glob

path = r'C:\\Users\\z0047cfr\\Project Work\\Factory Demand Model\\Input Data\\US plant capacity Utilization' # use your path
xls_files = glob.glob(path + "/*.xls")
xlsx_files = glob.glob(path + "/*.xlsx")
all_files = xls_files + xlsx_files

li = []

for filename in all_files:
    F2_1 = pd.read_excel(filename, index_col=None, sheet_name='Checkbox', skiprows = 3, skipfooter= 12)
    lst = ['Unnamed: 0', 'All Manufacturing  1/', 'NAICS 331', 'NAICS 332', 'NAICS 335', 'NAICS 336' ]
    #lst = ['Unnamed: 0', 'All Manufacturing  1/', 'NAICS 331']
    F2_1 = F2_1[F2_1.columns.intersection(lst)]
    F2_1 = F2_1.applymap(lambda x: x.strip() if isinstance(x, str) else x)  
    #F2_1=F2_1[1:]
    F2_1 = pd.melt(F2_1, id_vars=['Unnamed: 0'], value_vars=[ 'All Manufacturing  1/', 'NAICS 331', 'NAICS 332', 'NAICS 335', 'NAICS 336'])

    F2_1['Var_name'] = F2_1['Unnamed: 0'] + '_' + F2_1['variable']
    F2_1 = F2_1.drop( columns = ['Unnamed: 0','variable'])

    F2_1 = F2_1[['Var_name', 'value']]
    
    # since the data for each quarter is in a seperate file, the file name is used to identify quarter and input it in varname  
    year = filename[99:103]
    qtr = filename[120:122]
    qtr_yr = qtr + '_' + year 
    names = F2_1.columns.tolist()
    names[names.index('value')] = qtr_yr 
    F2_1.columns = names
    F2_1 = F2_1.dropna(subset=['Var_name'], how = 'any')
  
    
    li.append(F2_1) # appending the files into one list

#after seperate datasets are created for eachquarterly file, the below step will merge it together on the time period
F2_1_merged = reduce(lambda  left,right: pd.merge(left,right,on=['Var_name'],
                                            how='outer'), li)

#F2_1_merged=F2_1_merged[1:] # remove rows which have redundant values due to transpose (value all over ) - taken care later

F2_1_merged['Var_name'] = F2_1_merged.Var_name.str.replace('Manufacturing','mfg').str.replace('1/','').str.replace('expenditures','exp').str.replace('Machinery','Mach')
F2_1_merged =F2_1_merged[:-10] # remove last  columns since which come from unwanted data in raw data
F2_1_merged.shape
F2_1_merged.head(15)

,Var_name,q1_2014,q2_2014,q3_2014,q4_2014,q1_2015,q2_2015,q3_2015,q4_2015,q1_2016,...,q4_2017,q1_2018,q2_2018,q3_2018,q4_2018,q1_2019,q2_2019,q3_2019,q4_2019,q1_2020
0,ITEM 2/_All mfg,Value,Value,Value,Value,Value,Value,Value,Value,Value,...,Value,Value,Value,Value,Value,Value,Value,Value,Value,Value
1,Building capital exp_All mfg,3.5663,5.12705,2.13452,3.04379,2.73619,1.9,2.4,2,4.1,...,3.8,3,11.6,3.9,4.8,4.8,2.6,2.7,1.2,2.5
2,Mach capital exp_All mfg,12.2155,13.278,12.3091,8.84987,10.1556,10.2,12.5,10.4,11.2,...,9,11.4,9,10,13.3,14.3,13,12.4,10.3,10.5
3,Building retirements_All mfg,0.162434,0.267034,0.115594,0.16124,0.0832538,0.1,0.8,0.4,0.5,...,0.4,D,0.2,0.4,0.2,D,0.4,0.2,D,0
4,Mach retirements_All mfg,2.7742,2.24688,2.69464,2.22757,2.42695,0.9,3.2,1.2,2.7,...,1.1,1.7,1.4,3.2,1.3,1.3,2.8,1.7,3.4,1
5,"Price changed, product mix same_All mfg",17.5055,21.1699,20.1051,18.5703,18.1896,14.6,14.3,15.1,16.4,...,15.6,19.9,19,19.5,20.5,22.3,20.3,18.6,17.8,18.9
6,Revised estimation assumptions_All mfg,14.0197,15.3416,14.9898,13.1612,13.0912,15.3,15.2,13.5,12.1,...,11.5,11.9,15.9,13.5,12.7,14.5,15.1,10.5,13.6,11.8
7,Change in method of operation_All mfg,4.4562,7.06545,6.43835,5.13547,7.23247,6.6,6.2,5,7.1,...,5.7,5.7,7.1,5.1,2.4,3.5,5.2,4.2,4.8,6.6
8,Change in product mix or specifications_All mfg,27.9293,28.2933,29.9451,31.2237,31.1368,27.2,28.9,27.2,29.1,...,32.2,29.1,26.7,27.5,28.9,29.3,29.9,31.8,28.9,27
9,Change in material input_All mfg,4.59092,5.61383,4.14379,4.11436,5.15762,7,5.1,7.1,5.3,...,5.4,5.7,4.6,4.7,6,5.3,6.5,8.1,4.7,5.2


In [9]:
#Transpose
F2_1_=F2_1_merged.T
header=F2_1_.iloc[0]
F2_1_=F2_1_[1:]
F2_1_.columns=header
F2_1_ = F2_1_.add_prefix('Atrbn_Q_')
F2_1_.head(5)
#F2_1_.dtypes

Var_name,Atrbn_Q_ITEM 2/_All mfg,Atrbn_Q_Building capital exp_All mfg,Atrbn_Q_Mach capital exp_All mfg,Atrbn_Q_Building retirements_All mfg,Atrbn_Q_Mach retirements_All mfg,"Atrbn_Q_Price changed, product mix same_All mfg",Atrbn_Q_Revised estimation assumptions_All mfg,Atrbn_Q_Change in method of operation_All mfg,Atrbn_Q_Change in product mix or specifications_All mfg,Atrbn_Q_Change in material input_All mfg,...,Atrbn_Q_Environmental restrictions_All mfg,Atrbn_Q_Other (actual production less than full production)_All mfg,Atrbn_Q_Environmental restrictions_NAICS 331,Atrbn_Q_Other (actual production less than full production)_NAICS 331,Atrbn_Q_Environmental restrictions_NAICS 332,Atrbn_Q_Other (actual production less than full production)_NAICS 332,Atrbn_Q_Environmental restrictions_NAICS 335,Atrbn_Q_Other (actual production less than full production)_NAICS 335,Atrbn_Q_Environmental restrictions_NAICS 336,Atrbn_Q_Other (actual production less than full production)_NAICS 336
q1_2014,Value,3.5663,12.2155,0.162434,2.7742,17.5055,14.0197,4.4562,27.9293,4.59092,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
q2_2014,Value,5.12705,13.278,0.267034,2.24688,21.1699,15.3416,7.06545,28.2933,5.61383,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
q3_2014,Value,2.13452,12.3091,0.115594,2.69464,20.1051,14.9898,6.43835,29.9451,4.14379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
q4_2014,Value,3.04379,8.84987,0.16124,2.22757,18.5703,13.1612,5.13547,31.2237,4.11436,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
q1_2015,Value,2.73619,10.1556,0.0832538,2.42695,18.1896,13.0912,7.23247,31.1368,5.15762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# merge the three datasets and also work on time period to make it user friendly for time series statistics

F2 = F2_3_.merge(F2_2_, left_index = True, right_index = True).merge(F2_1_,left_index = True, right_index = True)
F2['period'] = F2.index
F2['DATE'] = F2.period.str.replace('_',' ').str.replace('q1','03 01').str.replace('q2','06 01').str.replace('q3','09 01').str.replace('q4','12 01')
F2['DATE'] =   pd.to_datetime(F2.DATE)
F2 = F2.drop( columns = ['period', 'month_x', 'month_y'])
F2 = F2.set_index('DATE')
F2 = F2.add_suffix('_Census')

F2.head()


,FU_Avg_Plant_hpw_all_mfg_Q_Census,FU_Avg_Plant_hpw_metal_mfg_Q_Census,FU_Avg_Plant_hpw_fab_metal_mfg_Q_Census,FU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q_Census,FU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q_Census,EU_Avg_Plant_hpw_all_mfg_Q_Census,EU_Avg_Plant_hpw_metal_mfg_Q_Census,EU_Avg_Plant_hpw_fab_metal_mfg_Q_Census,EU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q_Census,EU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q_Census,...,Atrbn_Q_Environmental restrictions_All mfg _Census,Atrbn_Q_Other (actual production less than full production)_All mfg _Census,Atrbn_Q_Environmental restrictions_NAICS 331_Census,Atrbn_Q_Other (actual production less than full production)_NAICS 331_Census,Atrbn_Q_Environmental restrictions_NAICS 332_Census,Atrbn_Q_Other (actual production less than full production)_NAICS 332_Census,Atrbn_Q_Environmental restrictions_NAICS 335_Census,Atrbn_Q_Other (actual production less than full production)_NAICS 335_Census,Atrbn_Q_Environmental restrictions_NAICS 336_Census,Atrbn_Q_Other (actual production less than full production)_NAICS 336_Census
DATE,,,,,,,,,,,,,,,,,,,,,
2014-03-01,70.6828,75.5845,65.8731,60.3044,56.2676,55.3486,65.7403,47.5543,45.8097,43.3403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-06-01,73.2892,74.6051,69.7745,65.4748,50.0679,58.0276,66.5139,50.3113,51.492,40.1535,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-09-01,73.5428,79.3307,61.4659,60.8504,65.6133,57.9359,72.0027,46.4948,47.6266,49.7006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014-12-01,71.298,73.8667,67.3321,62.517,48.2025,55.4041,65.6382,49.068,49.598,36.6632,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015-03-01,69.861,68.5114,68.6864,63.3704,51.4229,54.1936,62.1176,48.826,47.9389,39.0733,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
F2 = F2.mask(lambda x: x == 'D')
F2 = F2.mask(lambda x: x == 'Z')
F2 = F2.mask(lambda x: x == 'Value')
F2.head(1)
#F2.shape
#F2.dtypes

,FU_Avg_Plant_hpw_all_mfg_Q_Census,FU_Avg_Plant_hpw_metal_mfg_Q_Census,FU_Avg_Plant_hpw_fab_metal_mfg_Q_Census,FU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q_Census,FU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q_Census,EU_Avg_Plant_hpw_all_mfg_Q_Census,EU_Avg_Plant_hpw_metal_mfg_Q_Census,EU_Avg_Plant_hpw_fab_metal_mfg_Q_Census,EU_Avg_Plant_hpw_Metalwrk_mach_mfg_Q_Census,EU_Avg_Plant_hpw_railroad_roll_stk_mfg_Q_Census,...,Atrbn_Q_Environmental restrictions_All mfg _Census,Atrbn_Q_Other (actual production less than full production)_All mfg _Census,Atrbn_Q_Environmental restrictions_NAICS 331_Census,Atrbn_Q_Other (actual production less than full production)_NAICS 331_Census,Atrbn_Q_Environmental restrictions_NAICS 332_Census,Atrbn_Q_Other (actual production less than full production)_NAICS 332_Census,Atrbn_Q_Environmental restrictions_NAICS 335_Census,Atrbn_Q_Other (actual production less than full production)_NAICS 335_Census,Atrbn_Q_Environmental restrictions_NAICS 336_Census,Atrbn_Q_Other (actual production less than full production)_NAICS 336_Census
DATE,,,,,,,,,,,,,,,,,,,,,
2014-03-01,70.6828,75.5845,65.8731,60.3044,56.2676,55.3486,65.7403,47.5543,45.8097,43.3403,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# convert to numeric since all values are objects
#F2_ = F2.apply(pd.to_numeric,  errors='coerce') # there are some texts that need to be converted to missing.
F2_ = F2.apply(pd.to_numeric)

#F2_ = F2_.dropna(how = 'all')

F2_ = F2_.loc[:, F2_.isnull().mean() < 0.3] # remove columns with more than 30 percent data missing

#F2_.head(10)
#F2_.isna().sum().tail(20)
F2_.isna().sum()
F2_.shape
#F2_.dtypes

(25, 98)

In [13]:
 # fill missing values 
#F2_.isna().sum()
F2__ = F2_.resample('MS').first().fillna(method='bfill')
F2__.isna().sum()
F2__.head()
F2__.shape

(73, 98)